In [397]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

Grab the Excel file with all the translations of the TakeOne, and put it in the `_data` folder, renamed to `allChanges.xlsx`

In [398]:
# pandas uses the xlrd package by default, but xlrd has removed support for xlsx files
# thus we need to install and use openpyxl and set that to the engine used by read_excel()
source_df = pd.read_excel('allChanges.xlsx', engine = 'openpyxl')
source_df.rename(columns=lambda x: x.strip())


# source_df.columns = source_df.columns.str.replace(' ', '')
source_df = source_df.rename(columns = {
    'ENGLISH': 'en',
    'SPANISH': 'es',
    'CHINESE': 'zh-TW',
    'KOREAN': 'ko',
    'VIETNAMESE': 'vi',
    'JAPANESE': 'ja',
    'RUSSIAN': 'ru',
    'ARMENIAN': 'hy'
})

# source_df

Create a dataframe with a subset of the data (just the line change details).

Manually set the range of rows to copy.

In [399]:
line_changes_df = source_df.iloc[18:]

# line_changes_df

Manually adjust the data as needed.

In [400]:
# Combine the last two rows: 55, 56
# The row indices are different when accessing them as a Series versus the DataFrame

# for column in line_changes_df:
#     colSeries = line_changes_df[column]
#     combinedText = colSeries.values[37] + colSeries.values[38]
#     line_changes_df.at[55, colSeries.name] = combinedText
#     line_changes_df.at[56, colSeries.name] = ''

# line_changes_df = line_changes_df.drop([56])

line_changes_df = line_changes_df.reset_index(drop=True)

In [401]:
# Split the based on the em-dash
# Create a new column 'label' that uses the first value from the split
# Note that this value may be a line number or may be a text description

line_changes_df['label'] = line_changes_df['en'].str.split('–').str[0].str.strip()


In [402]:
# Remove line numbers from details

for column in line_changes_df:
    for i in range(len(line_changes_df[column])):
        if '–' in line_changes_df.at[i, column]:
            line_changes_df.at[i, column] = line_changes_df.at[i, column].split('–')[1].strip()


In [403]:
# Duplicate rows that have two line numbers

def separate_rows(df, col, delim):
    colSeries = df[col]
    for item in colSeries:
        if delim in item:
            routes = item.split(delim)
            matching_df = df.loc[df[col] == item]

            matching_df = matching_df.append(matching_df)
            matching_df = matching_df.reset_index(drop=True)
            
            matching_df.at[0, col] = routes[0].strip()
            matching_df.at[1, col] = routes[1].strip()
            
            df = df[df[col] != item]
            df = df.append(matching_df)
            df = df.reset_index(drop=True)
    return df

line_changes_df = separate_rows(line_changes_df, 'label', '/')


In [404]:
# Create a new column 'line' that uses the value from column 'label' only if it is a digit
def check_digit(x):
    if x['label'].isdigit():
        return x['label']
    else:
        return 0

line_changes_df['line'] = 0
line_changes_df['line'] = line_changes_df.apply(check_digit, axis=1)

# Set specific instance of the C & K Line Link, which is actually route number 857

line_changes_df.loc[line_changes_df.label == 'C & K Line Link', 'line'] = 857


In [405]:
# Read in full list of lines as pulled from the current GTFS routes.txt file (slightly modified)

routes_df = pd.read_csv('routes.csv', dtype={'line': str, 'order': int})
routes_df['line'] = routes_df['line'].astype(str)
routes_df['line'].apply(lambda x: x.strip())

# routes_df.count()

0        2
1        4
2       10
3       14
4       16
5       18
6       20
7       28
8       30
9       33
10      35
11      37
12      38
13      40
14      45
15      48
16      51
17      53
18      55
19      60
20      62
21      66
22      70
23      76
24      78
25      81
26      90
27      92
28      94
29      96
30     102
31     105
32     106
33     108
34     110
35     111
36     115
37     117
38     120
39     125
40     127
41     128
42     150
43     152
44     154
45     155
46     158
47     161
48     162
49     164
50     165
51     166
52     167
53     169
54     177
55     179
56     180
57     182
58     202
59     204
60     205
61     206
62     207
63     209
64     210
65     211
66     212
67     215
68     217
69     218
70     222
71     224
72     230
73     232
74     233
75     234
76     236
77     237
78     240
79     242
80     243
81     244
82     246
83     251
84     256
85     258
86     260
87     265
88     266
89     267
90     268

In [406]:
# Merge the two dataframes using an outer join

line_changes_df['line'] = line_changes_df['line'].astype(str)
routes_and_line_changes_df = pd.merge(line_changes_df, routes_df, on='line', how='outer')

routes_and_line_changes_df = routes_and_line_changes_df.sort_values(by='order')

# confirm we have the correct final counts of lines with changes and total number of lines
# routes_and_line_changes_df.count()

In [407]:
# check to see if any lines have no order value attached
routes_and_line_changes_df.loc[routes_and_line_changes_df['order'].isna()]

,en,es,zh-TW,hy,ko,vi,ja,ru,label,line,order


In [408]:
# add the PDFs
import os

#define the folders to look through
# folders = os.listdir("../files/schedules")

#set an array for the file types
timetables_list = []

#create a list of file types
for root, dirs, files in os.walk("../files/schedules"):
    for filename in files:
        lines = filename.replace(" ","_").split("_TT")[0].split("-")
        for line in lines:
            this_schedule = {}
            this_schedule['line'] = line.lstrip("0")
            this_schedule['newSchedule'] = "/files/schedules/"+filename
            timetables_list.append(this_schedule)

schedule_df = pd.DataFrame(timetables_list)
# schedule_df.tail(10)

In [409]:
# merge schedules with list

new_pdfs_and_routes_and_line_changes_df = pd.merge(routes_and_line_changes_df, schedule_df, on='line', how='outer')
# new_pdfs_and_routes_and_line_changes_df

# the 134 is a new line.  the 807 is the new crenshaw line

In [410]:
# pull current schedule PDFs from the metro.net WP REST API


wp_url = 'https://www.metro.net/wp-json/wp/v2/line-override'
wp_df = pd.read_json(wp_url)

wp_df = pd.DataFrame(wp_df['acf'])
# wp_df


In [411]:
# Grab the line-override entries from WordPress that have a pdf_file.url value

def get_urls(x):
    temp_df = pd.json_normalize(x['acf'])
    if 'pdf_file.url' in temp_df.columns:
        return temp_df['pdf_file.url']
    return

wp_df_urls = wp_df.apply(get_urls, axis=1)

wp_df_urls = wp_df_urls.dropna()

wp_df_urls = wp_df_urls.rename(columns={0: "currentSchedule"})

# wp_df_urls


In [412]:

def get_no_urls(x):
    temp_df = pd.json_normalize(x['acf'])
    if 'pdf_file.url' not in temp_df.columns:
        return temp_df['line_id']
    return

wp_df_no_urls = wp_df.apply(get_no_urls, axis=1)

wp_df_no_urls = wp_df_no_urls.dropna()

wp_df_no_urls

Series([], dtype: object)

In [413]:
# Grab the line and add it to a separate column

def get_lines(x):
    split_url = x['currentSchedule'].split('/')
    filename = split_url[len(split_url)-1].replace(' ', '_')
    line = ''

    if '_TT' in filename:
        line = filename.split('_TT')[0].lstrip('0')
    elif '-TT' in filename:
        line = filename.split('-TT')[0].lstrip('0')
    elif '.pdf' in filename:
        line = filename.split('_')[0].lstrip('0')
    else:
        line = filename
    x['line'] = line
    return

current_pdfs_list = []

wp_df_urls['line'] = ''
wp_df_urls.apply(get_lines, axis=1)

# wp_df_urls

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
33     None
34     None
35     None
36     None
37     None
38     None
39     None
40     None
41     None
42     None
43     None
44     None
45     None
46     None
47     None
48     None
49     None
50     None
51     None
52     None
53     None
54     None
55     None
56     None
57     None
58     None
59     None
60     None
61     None
62     None
63     None
64     None
65     None
66     None
67     None
68     None
69     None
70     None
71     None
72     None
73     None
74     None
75     None
76     None
77     None
78     None
79     None
80     None
81     None
82     None
83  

In [414]:
# wp_df_urls = separate_rows(wp_df_urls, 'line', '_')

wp_df_urls = wp_df_urls.drop_duplicates(subset=['line'])

colSeries = wp_df_urls['line']

for item in colSeries:
    if '-' in item:
        routes = item.split('-')
        matching_df = wp_df_urls.loc[wp_df_urls['line'] == item]
        matching_df = matching_df.append(matching_df)
        matching_df = matching_df.reset_index(drop=True)

        matching_df.at[0, 'line'] = routes[0].strip().lstrip('0')
        matching_df.at[1, 'line'] = routes[1].strip().lstrip('0')

        wp_df_urls = wp_df_urls[wp_df_urls['line'] != item]
        wp_df_urls = wp_df_urls.append(matching_df)
        wp_df_urls = wp_df_urls.reset_index(drop=True)

# wp_df_urls


In [415]:
# combine current schedules 

details_df = pd.merge(new_pdfs_and_routes_and_line_changes_df, wp_df_urls, on='line', how='left')


In [416]:
details_df.loc[details_df['currentSchedule'].isna()]


,en,es,zh-TW,hy,ko,vi,ja,ru,label,line,order,newSchedule,currentSchedule
118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134,NaN,/files/schedules/134_TT_10-23-22.pdf,NaN


In [417]:
# manual adjustment for line 534, which is being renamed to the 134

details_df.loc[details_df['line'] == '534', 'newSchedule'] = details_df.loc[details_df['line'] == '134', 'newSchedule'].values[0]

details_df.loc[details_df['line'] == '534']

details_df = details_df[details_df['line'] != '134']

# details_df.tail(10)

In [418]:
# manual filling in of label field

def apply_line_labels(row):
    if pd.isna(details_df.at[row.name, 'label']):
        if details_df.at[row.name, 'line'] == '807':
            details_df.at[row.name, 'label'] = 'K Line'
        elif details_df.at[row.name, 'line'] == '901':
            details_df.at[row.name, 'label'] = 'G Line (Orange)'
        elif details_df.at[row.name, 'line'] == '950' or details_df.at[row.name, 'line'] == '910':
            details_df.at[row.name, 'label'] = 'J Line (Silver)'
        else:
            details_df.at[row.name, 'label'] = details_df.at[row.name, 'line']
    return

details_df.apply(apply_line_labels, axis=1)



0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
33     None
34     None
35     None
36     None
37     None
38     None
39     None
40     None
41     None
42     None
43     None
44     None
45     None
46     None
47     None
48     None
49     None
50     None
51     None
52     None
53     None
54     None
55     None
56     None
57     None
58     None
59     None
60     None
61     None
62     None
63     None
64     None
65     None
66     None
67     None
68     None
69     None
70     None
71     None
72     None
73     None
74     None
75     None
76     None
77     None
78     None
79     None
80     None
81     None
82     None
83  

In [419]:
details_df.tail(10)

,en,es,zh-TW,hy,ko,vi,ja,ru,label,line,order,newSchedule,currentSchedule
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,686,686,110.0,NaN,https://cdn.beta.metro.net/wp-content/uploads/2022/09/07162823/686_TT_06-27-21.pdf
110,"Line 224 will extend from Sylmar Station to Olive View Medical Center in place of Line 690, which will continue to operate its usual route between Olive View Medical Center and Sunland. Line 690 frequency between Olive View Medical Center and Sunland will increase from every 50 minutes to every 30 minutes weekdays, and 40 minutes Saturdays and Sundays.","La Línea 224 se extenderá desde la estación Sylmar hasta Olive View Medical Center en lugar de la Línea 690, que continuará operando su ruta habitual entre Olive View y Sunland. La frecuencia de la Línea 690 entre Olive View Medical Center y Sunland aumentará de cada 50 minutos a cada 30 minutos entre semana y 40 minutos los sábados y domingos.",224號線將取代690號線，從Sylmar站延 伸至Olive View Medical Center，而690線路將繼 續在Olive View Medical Center和Sunland之間 的常規線路運行。Olive View Medical Center和 Sunland之間的690號線的班次間隔時間將從週 間每50分鐘一班增加到每30分鐘一班，週六和 週日增加到每40分鐘一班。,"224 գիծը կձգվի Sylmar Station կայարանից մինչև Olive View բժշկական կենտրոն 690 գծի փոխարեն, որը կշարունակի գործարկել իր սովորական երթուղին Olive View բժշկական կենտրոն-ի և Sunland-ի միջև: 690 գծի հաճախականությունը Olive View բժշկական կենտրոն-ի և Sunland-ի միջև կաճի յուրաքանչյուր 50 րոպեից մինչև յուրաքանչյուր 30 րոպե աշխատանքային օրերին և 40 րոպե՝ շաբաթ և կիրակի օրերին:","224번 노선은 Olive View Medical Center 와 Sunland 사이 평소 노선을 계속 운행할 690 번 노선을 대체해 Sylmar에서 Olive View Medical Center까지 연장 운행합니다. 690번 노선의 Olive View Medical Center 와 Sunland 사이 운행 간격은 매 50분에서 평일 30분, 토요일과 일요일 40분으로 짧아집니다.","Tuyến 224 sẽ kéo dài từ Ga Sylmar đến Olive View Medical Center thay cho Tuyến 690, tuyến này sẽ tiếp tục khai thác lộ trình thông thường từ Olive View Medical Center đến Sunland. Tần suất Tuyến 690 từ Olive View Medical Center đến Sunland sẽ tăng từ 50 phút một chuyến thành 30 phút một chuyến các ngày trong tuần và 40 phút một chuyến các ngày thứ Bảy và Chủ Nhật.",224系統は、690系統の代わりに、Symar駅 からOlive View Medical Centerまで延長して運 行します。690系統は、Olive View Medical Center とSunlandの間は通常通りのルートで引き続 き運行します。690系統のOlive View Medical Center とSunland間の運行頻度は、50分に1本 が、平日は30分ごとに、土日は40分ごとに、本数 が増えます。,"Маршрут автобуса №224 будет продлен от остановки Sylmar до Olive View Medical Center и заменит маршрут № 690, автобусы которого продолжат движение по своему обычному маршруту между Olive View Medical Center и Sunland. Промежуток между рейсами автобуса № 690 между Olive View Medical Center и Sunland будет уменьшен с 50 минут до 30 минут в будни и до 40 минут по субботам и воскресеньям.",690,690,111.0,/files/schedules/690_TT_10-23-22.pdf,https://cdn.beta.metro.net/wp-content/uploads/2022/06/24094824/690_TT_06-26-22.pdf
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,720,720,112.0,/files/schedules/720_TT_10-23-22.pdf,https://cdn.beta.metro.net/wp-content/uploads/2022/06/24094948/720_TT_06-26-22.pdf
112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,754,754,113.0,/files/schedules/754_TT_10-23-22.pdf,https://cdn.beta.metro.net/wp-content/uploads/2022/06/24095134/754_TT_06-26-22.pdf
113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,761,761,114.0,NaN,https://cdn.beta.metro.net/wp-content/uploads/2022/06/24095246/761_TT_06-26-22.pdf
114,New bus line linking new K Line Westchester/Veterans Station and the existing C Line (Green) Aviation/LAX Station.,Una nueva línea de autobús para conectar la nueva estación Crenshaw K Line Westchester/Veterans y la estación actual C Line Aviation/LAX.,新設的巴士線路將新的 Crenshaw K Line Westchester/Veteran站和現有 的C線（綠線）Aviation/LAX 站加以連結。,"Նոր գիծ, որը կապում է Crenshaw K Line Westchester/Veteran նոր կայարանը և C Line (Green) Aviation/LAX գործող կայարանը:",새로운 Crenshaw K 라인 Westchester/Veteran 역과 기존 C 라인(그린) Aviation/LAX 역을 연결하는 새로운 버스 노선이 생겼습니다.,Tuyến xe buýt mới nối Ga Crenshaw K Line Westchester/Veteran mới và Ga C Line (Green) Aviation/LAX hiện tại.,新しいCrenshaw K Line Westchester/Veteran 駅と既存のC Line

In [420]:
summary_df = source_df.loc[5:15]

for column in summary_df:
    colSeries = summary_df[column]
    combinedText = '<p>' + colSeries.values[1] + '</p><p>' + colSeries.values[2] + '</p>'
    summary_df.at[6, colSeries.name] = combinedText

    combinedText = '<p>' + colSeries.values[4] + '</p><p>' + colSeries.values[5] + '</p><p>' + colSeries.values[6] + '</p>'
    summary_df.at[9, colSeries.name] = combinedText

    combinedText = '<p>' + colSeries.values[8] + '</p><p>' + colSeries.values[9] + '</p><p>' + colSeries.values[10] + '</p>'
    summary_df.at[13, colSeries.name] = combinedText

summary_df = summary_df.drop([7, 10, 11, 14, 15])
summary_df = summary_df.reset_index(drop=True)


In [421]:
intro_df = source_df.loc[[0,4]]

def remove_sentence(row):
    if row.name == 4:
        for col in intro_df.columns:
            if ':' in row[col]:
                intro_df.at[4, col] = row[col].split(':')[0].replace('"', '') + ':'
            elif '。' in row[col]:
                intro_df.at[4, col] = row[col].split('。')[0] + '。'
            else:
                if col == 'ru':
                    intro_df.at[4, col] = row[col].split('.')[0] + '. ' + row[col].split('.')[1] + '.'
                else:
                    intro_df.at[4, col] = row[col].split('.')[0] + '. '
    return

intro_df.apply(remove_sentence, axis=1)

intro_df = intro_df.reset_index(drop=True)



In [422]:
#set an array for the file types
language_list = {
    'English': {
        'label': 'English',
        'order': 0
    },
    'Spanish': {
        'label': 'Español (Spanish)',
        'order': 1
    },
    'Chinese': {
        'label': '中文 (Chinese Traditional)',
        'order': 2
    },
    'Korean': {
        'label': '한국어 (Korean)',
        'order': 3
    },
    'Vietnamese': {
        'label': 'Tiếng Việt (Vietnamese)',
        'order': 4
    },
    'Japanese': {
        'label': '日本語 (Japanese)',
        'order': 5
    },
    'Russian': {
        'label': 'русский (Russian)',
        'order': 6
    },
    'Armenian': {
        'label': 'Армянский (Armenian)',
        'order': 7
    }
}
    
takeones_list = {}

#create a list of file types
for root, dirs, files in os.walk("../files/takeones"):
    for filename in files:
        lang = filename.split(' - ')[1].split('.')[0]
        takeones_list[language_list[lang]['order']] = {
            'text': language_list[lang]['label'],
            'url': '/files/takeones/' + filename
        }
        
takeones_list

takeones_dict = {
    'takeones': {
        'description': 'Download a PDF version of this page in ',
        'files': takeones_list
        }
}


In [423]:
# details_df.to_json('lineChangesOctober.json', orient='records')

details_df = details_df.fillna('')
langs = ['en','es','zh-TW','ko','vi','ja','ru','hy']
all_langs_results_arr = []

for lang in langs:
    ### Intro Section
    temp_intro_df = intro_df[lang]
    temp_intro_df = temp_intro_df.rename(index={0: 'header', 1: 'details'})

    temp_intro_dict = temp_intro_df.to_dict()
    temp_intro_dict = {
        'intro': temp_intro_dict
    }

    ### Summary Section
    temp_summary_df = summary_df[[lang]]
    temp_summary_df = temp_summary_df.rename(columns={lang: 'description'})

    for i in range(len(temp_summary_df['description'])):
        if i % 2 == 0:
            temp_summary_df.loc[i, 'details'] = temp_summary_df.loc[i+1, 'description']
        else:
            temp_summary_df = temp_summary_df.drop(i)

    temp_summary_dict = temp_summary_df.to_dict(orient='records')
    temp_summary_df = pd.DataFrame({'summary': temp_summary_dict})

    temp_summary_dict = temp_summary_df.to_dict(orient='list')

    ### Details Section
    temp_details_df = details_df[[lang, 'line', 'label', 'newSchedule', 'currentSchedule']]
    temp_details_df = temp_details_df.rename(columns={lang: 'description'})
    temp_details_dict = temp_details_df.to_dict(orient='records')

    temp_details_df = pd.DataFrame({'details': temp_details_dict})
    temp_details_dict = temp_details_df.to_dict(orient='list')

    # use the dictionary unpacking notation: **
    # to create a new dictionary comprised of the key/value pairs from the other dictionaries
    result_dict = temp_intro_dict
    result_dict.update(takeones_dict)
    result_dict.update(temp_summary_dict)
    result_dict.update(temp_details_dict)
    
    all_langs_results_arr.append({
        'lang': lang,
        'content': result_dict
    })

# print(all_langs_results_dict)


In [424]:
import json

# all_langs_results_json = json.dumps(all_langs_results_dict)
# print(all_langs_results_json)

with open('../data/allChanges.json', 'w') as json_file:
    json.dump(all_langs_results_arr, json_file)